<a href="https://colab.research.google.com/github/mistryvivek/YRKCS-PRBX/blob/main/Model_1a_Basic_NN_(Earth_movers_distance).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import OneHotEncoder

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


# Setup Weights and Bias.

* AI experimental tool.

In [3]:
!pip install wandb -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 48.7 MB/s eta 0:00:00


In [4]:
# Log in to your W&B account
import wandb
import random
import math

In [5]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Earth Movers Distance Loss

* https://github.com/meder411/PyTorch-EMDLoss

In [11]:
import torch.nn.functional as F

# EMD loss function
def EMD(preds, targets):
    # Normalize the predictions and targets to probability distributions (summing to 1)
    preds = F.softmax(preds, dim=-1)  # Apply softmax to get probabilities
    targets = F.softmax(targets, dim=-1)

    # Compute the Earth Mover's Distance
    # Here, we use a simple version that computes the L1 norm (Manhattan distance) between the distributions
    distance = torch.abs(preds - targets).sum(dim=-1)

    return distance.mean()  # Return the average distance over the batch

## Modify code for sliding window approach (like wk11 code).

* Look at the full race image instead.
* What data types are in the wk11 practical.
* No vocab size required because they apply to catogorical contexts.

In [6]:
BASE_PATH = r"/content/drive/MyDrive/prbx_data/v1/"
max_race_size = 0
max_tyre_life = 0

In [7]:
# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1]else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)

            training_inputs.append(tyre_life_array)
            training_outputs.append(stint_change_array)

In [8]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1]else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)

            testing_inputs.append(tyre_life_array)
            testing_outputs.append(stint_change_array)

In [9]:
class RNNv1b(nn.Module):
    def __init__(self,input_size,hidden_size): #vocab_size
        super(RNNv1b, self).__init__()
        #self.embedding =  nn.Embedding(vocab_size, hidden_size)
        self.linear_x = nn.Linear(input_size, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,1)
        self.tanh = nn.Tanh()

    def forward(self,x,hprev):
        """
        h = self.tanh(self.embedding(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        """
        h = self.tanh(self.linear_x(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        return h,y

In [17]:
def calculate_loss(model,inputs,targets,hprev,vocab_size):
  loss_func = EMD
  seq_length = len(inputs)
  outputs = []
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(torch.tensor([inputs[t]], device=device),hprev)
    # Gradually build up matrix of output logits of size seq_length * vocab_size - we want it at every time step do not want this.
    # Compute cross entropy loss for seq_length actual targets against estimated distributions
    # RuntimeError: result type Float can't be cast to the desired output type Long
    outputs.append(y)

  outputs = torch.stack(outputs).squeeze(1)
  loss = loss_func(outputs ,targets)

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

In [25]:
# Model parameters to be logged by w and b.
hidden_size = 10
lr = 0.0001
iterations = 5000
input_parameters = ['TyreLife']
dataset = 'v1'
#vocab_size = int(max_tyre_life) + 1 # Possiblity for each tyre_life
input_size = 1

In [26]:
# PADDING
def add_padding(max_race_size, sequence):
    if len(sequence) >= max_race_size:
        return sequence[:max_race_size]
    else:
        padding = [0] * (max_race_size - len(sequence))
        return np.concatenate([sequence,padding])

In [27]:
def check_accuracy():
  sigmoid = nn.Sigmoid()
  total, matches = 0, 0
  no_pit_prediction_total, correct_no_pit_prediction = 0,0
  pit_prediction_total, correct_pit_prediction = 0,0

  for t in range(len(testing_inputs) - 1):
      inputs = add_padding(int(max_race_size),testing_inputs[t])
      outputs = add_padding(int(max_race_size),testing_outputs[t])
      hprev = torch.zeros(hidden_size, device=device)

      for x in range(len(inputs) - 1):

        hprev, y = model(torch.tensor([inputs[x]], device=device, dtype=torch.float32), hprev)
        y = sigmoid(y.squeeze(0))

        if (y.item() > 0.5) == (outputs[x] == 1.0):
          matches += 1

        if (y.item() > 0.5):
          pit_prediction_total += 1
          if 1 in outputs[max(0, x - 2): min(int(max_race_size), x+4)]:
              correct_pit_prediction += 1
        else:
          no_pit_prediction_total += 1
          if y.floor().item() == outputs[x]:
              correct_no_pit_prediction += 1

        total += 1

  if total != 0:
    print(f"Total Accuracy: {matches / total * 100}")
  else:
    print("Total Accuracy: 0.0")

  if no_pit_prediction_total != 0:
    print(f"Total No Pit Accuracy: {correct_no_pit_prediction / no_pit_prediction_total * 100}")
  else:
    print(f"Total No Pit Accuracy: 0.0")

  if pit_prediction_total != 0:
    print(f"Total Pit Accuracy: {correct_pit_prediction / pit_prediction_total * 100}")
  else:
    print(f"Total Pit Accuracy: 0.0")

  print(f"Total Pit Predictions: {pit_prediction_total}")

In [28]:
model = RNNv1b(input_size,hidden_size)#vocab_size
model.to(device)

optim = torch.optim.Adam(model.parameters(), lr = lr)

n, p = 0, 0
while n<=iterations:
  hprev = torch.zeros(hidden_size, device=device) # reset RNN memory

  # Why apply padding here
  inputs = add_padding(int(max_race_size),training_inputs[p])
  outputs = add_padding(int(max_race_size),training_outputs[p])

  # Extract next subsequence of characters
  inputs = torch.tensor(training_inputs[p],dtype=torch.float32, device=device)
  targets = torch.tensor(training_outputs[p],dtype=torch.float32, device=device)

  # Compute loss for current subsequence
  loss, hprev = calculate_loss(model,inputs,targets,hprev,input_size)
  optim.zero_grad()
  loss.backward()
  optim.step()

  if n % 500 == 0:
    print('[{:}] Loss: {:.2f}'.format(n,loss.item()))
    check_accuracy()

  p += 1 # move data pointer
  n += 1 # iteration counter

  if p == int(max_race_size) - 1:
    p = 0

[0] Loss: 0.21
Total Accuracy: 3.1888020446372853
Total No Pit Accuracy: 97.10982658959537
Total Pit Accuracy: 14.407336599354762
Total Pit Predictions: 33476
[500] Loss: 0.09
Total Accuracy: 2.7043894320782194
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14.493744242027995
Total Pit Predictions: 33649
[1000] Loss: 0.01
Total Accuracy: 2.7043894320782194
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14.493744242027995
Total Pit Predictions: 33649
[1500] Loss: 0.17
Total Accuracy: 2.7043894320782194
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14.493744242027995
Total Pit Predictions: 33649
[2000] Loss: 0.15
Total Accuracy: 2.7043894320782194
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14.493744242027995
Total Pit Predictions: 33649
[2500] Loss: 0.07
Total Accuracy: 2.7043894320782194
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14.493744242027995
Total Pit Predictions: 33649
[3000] Loss: 0.10
Total Accuracy: 2.7043894320782194
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14